In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
import scipy.optimize as optimize
import requests

In [2]:
# Import arps decline and discounted cash flow functions
# Will change in the future to a python library
fcst_url = 'https://raw.githubusercontent.com/jshumway0475/Petroleum/main/prod_fcst_functions.py'
dcf_url = 'https://raw.githubusercontent.com/jshumway0475/Petroleum/main/dcf_functions.py'
response1 = requests.get(fcst_url)
response2 = requests.get(dcf_url)
exec(response1.text)
exec(response2.text)

In [3]:
# Import data for decline parameters
prop_list_url = r'https://raw.githubusercontent.com/jshumway0475/Petroleum/main/property_table.csv'
prop_list = pd.read_csv(prop_list_url)
prop_list = prop_list.query('INCLUDE == 1')
prop_list = prop_list.fillna(0)
prop_list.reset_index(drop = True, inplace = True)
R = prop_list.index # Rows

In [4]:
# Loop through DataFrame and output monthly oil volumes
# type: ignore
duration = 600

oil = lambda w: arps_segments(
    prop_list.loc[w, 'PROPNUM'],
    1,
    prop_list.loc[w, 'OIL_IP'],
    prop_list.loc[w, 'OIL_IP2'], 
    prop_list.loc[w, 'OIL_IP3'],
    prop_list.loc[w, 'OIL_DI'] / 100,
    prop_list.loc[w, 'Oil_Def'] / 100, 
    prop_list.loc[w, 'OIL_B'],
    prop_list.loc[w, 'OIL_SEG1_TIME'],
    prop_list.loc[w, 'OIL_SEG2_TIME'], 
    duration
)
v_oil = np.vectorize(oil, otypes = [object])
oil_nparr = v_oil(R)

# Loop through DataFrame and output monthly gas volumes

gas = lambda w: arps_segments(
    prop_list.loc[w, 'PROPNUM'],
    2,
    prop_list.loc[w, 'GAS_IP'],
    prop_list.loc[w, 'GAS_IP2'], 
    prop_list.loc[w, 'GAS_IP3'],
    prop_list.loc[w, 'GAS_DI'] / 100,
    prop_list.loc[w, 'GAS_Def'] / 100, 
    prop_list.loc[w, 'GAS_B'],
    prop_list.loc[w, 'GAS_SEG1_TIME'],
    prop_list.loc[w, 'GAS_SEG2_TIME'], 
    duration
)
v_gas = np.vectorize(gas, otypes = [object])
gas_nparr = v_gas(R)

In [5]:
# Import price files
price_file_url = r'https://raw.githubusercontent.com/jshumway0475/Petroleum/main/STR-071521.csv'
BaseDate = '2021-09'
MaxDate = pd.to_datetime(prop_list['Start_Date']).max()
MaxDate_np = MaxDate.to_datetime64()
add_months = int(round((MaxDate - np.datetime64(BaseDate)) / np.timedelta64(1, 'M'), 0)) + 1
str_periods = duration + add_months
dates = pd.DataFrame(pd.date_range(BaseDate, periods = str_periods, freq = 'MS'), columns = ['Date'])
strip_price = pd.read_csv(price_file_url)
strip_price['Date'] = pd.to_datetime(strip_price['Date'])

# Create diff array
pepl = {
    BaseDate: -0.137,
    '01/2022': -0.17,
    '01/2023': -0.249,
    '01/2024': -0.26,
    '01/2025': -0.214,
    '01/2026': -0.212
}
pepl_pd = pd.DataFrame(pepl.items(), columns = ['Date', 'paj_gas'])
pepl_pd['Date'] = pepl_pd['Date'].astype('datetime64[ns]')

# Create numpy arrays for cash flow calcs
strip_price = pd.merge(dates, strip_price, how = 'left', on = 'Date')
strip_price = pd.merge(strip_price, pepl_pd, how = 'left', on = 'Date')
strip_price.fillna(method = 'ffill', inplace = True)
oil_price_full = np.transpose(strip_price[['OilPrice']].to_numpy())[0]
gas_price_full = np.transpose(strip_price[['GasPrice']].to_numpy())[0]
pepl_diff_full = np.transpose(strip_price[['paj_gas']].to_numpy())[0]

In [6]:
# Generate price and volume arrays with proper indexing
# type: ignore

def oil_price(x):
    # Create shift integer for arrays
    StartDate = prop_list.loc[x, 'Start_Date']
    DateDiff = month_diff(BaseDate, StartDate)
    
    # Shift price arrays
    uid = np.full((duration + DateDiff,), oil_nparr[x][0][0])
    oil_pri = oil_price_full[:duration + DateDiff]
    oil_price = np.vstack((uid, oil_pri))
    return oil_price

voil_price = np.vectorize(oil_price, otypes = [object])                         
oilprice = voil_price(R)

def gas_price(x):
    # Create shift integer for arrays
    StartDate = prop_list.loc[x, 'Start_Date']
    DateDiff = month_diff(BaseDate, StartDate)
    
    # Shift price arrays
    uid = np.full((duration + DateDiff,), gas_nparr[x][0][0])
    gas_pri = gas_price_full[:duration + DateDiff]
    gas_price = np.vstack((uid, gas_pri))
    return gas_price

vgas_price = np.vectorize(gas_price, otypes = [object])                         
gasprice = vgas_price(R)

def gas_diff(x):
    # Create shift integer for arrays
    StartDate = prop_list.loc[x, 'Start_Date']
    DateDiff = month_diff(BaseDate, StartDate)
    
    # Shift price arrays
    uid = np.full((duration + DateDiff,), gas_nparr[x][0][0])
    pepl = pepl_diff_full[:duration + DateDiff]
    gas_diff = np.vstack((uid, pepl))
    return gas_diff

vgas_diff = np.vectorize(gas_diff, otypes = [object])                         
gasdiff = vgas_diff(R)

def volarray(x):
    uid = oil_nparr[x][0]
    StartDate = prop_list.loc[x, 'Start_Date']
    DateDiff = month_diff(BaseDate, StartDate)
    
    # create well specific volume arrays
    month = np.round(oil_nparr[x][2] + DateDiff - 1, 0)
    oil_vol = np.round(oil_nparr[x][6], 4)
    gas_vol = np.round(gas_nparr[x][6], 4)
    nvol_np = np.vstack((uid, month, oil_vol, gas_vol))
    prop = np.full((DateDiff,), uid[0])
    delay = np.arange(DateDiff)
    oil_zeros = np.zeros((DateDiff),)
    gas_zeros = oil_zeros
    shift = np.vstack((prop, delay, oil_zeros, gas_zeros))
    vol_arr = np.column_stack((shift, nvol_np))
    
    return vol_arr

vvolarray = np.vectorize(volarray, otypes = [object])
vol_np = vvolarray(R)

In [8]:
# Generate full monthly cash flow arrays
# type: ignore
# define constant input parameters
eloss = 0
weight = 1.0
prod_wt = 1.0
inv_wt = 1.0
stx_oil = 0.0795
stx_gas = 0.0795
stx_ngl = 0.0795
adval = 0
aban = 150000

# Create function for slicing the volume array and calculating the monthly cash flow
def econ_ncf_iter(r):    
    econ_ncf_iter = econ_cf(
        index = r,
        uid = prop_list.loc[r, 'PROPNUM'],
        wi = prop_list.loc[r, 'WI'], 
        nri = prop_list.loc[r, 'NRI'],
        roy = prop_list.loc[r, 'Royalty'],
        eloss = eloss, 
        weight = weight,
        prod_wt = prod_wt,
        inv_wt = inv_wt, 
        shrink = np.round(prop_list.loc[r, 'SHRINK'] / 100, 6), 
        btu = np.round(prop_list.loc[r, 'BTU'] / 1000, 6), 
        ngl_yield = np.round(prop_list.loc[r, 'NGL/GAS'], 6), 
        pri_oil = np.extract(oilprice[r][0] == prop_list.loc[r, 'PROPNUM'], oilprice[r][1]),
        pri_gas = np.extract(gasprice[r][0] == prop_list.loc[r, 'PROPNUM'], gasprice[r][1]),
        paj_oil = prop_list.loc[r, 'PAJ_OIL'], 
        paj_gas = np.extract(gasdiff[r][0] == prop_list.loc[r, 'PROPNUM'], gasdiff[r][1]), 
        paj_ngl = prop_list.loc[r, 'PAJ_NGL'],
        stx_oil = stx_oil,
        stx_gas = stx_gas,
        stx_ngl = stx_ngl,
        adval = adval,
        opc_fix = np.round(prop_list.loc[r, 'OPC/T'], 2), 
        opc_oil = np.round(prop_list.loc[r, 'OIL_OPEX'], 2), 
        opc_gas = np.round(prop_list.loc[r, 'GAS_OPEX'], 2), 
        capex = np.round(prop_list.loc[r, 'CAPITAL'] * 1000, 2),
        aban = aban,
        volumes = vol_np
    )
    return econ_ncf_iter

# generate net cash flow array
econ_ncf = lambda r: econ_ncf_iter(r)
vecon_ncf = np.vectorize(econ_ncf_iter, otypes = [object])
ncf_arr_packed = vecon_ncf(R)

In [9]:
ncf_pd_dflist = []
columns = [
    'UID',
    'Month',
    'Grs Oil',
    'Grs Gas',
    'Net Oil',
    'Net Gas',
    'Net NGL',
    'Oil Revenue',
    'Gas Revenue', 
    'NGL Revenue',
    'Total Revenue',
    'Total Tax',
    'OPEX',
    'Operating Income',
    'Cumulative Op CF',
    'Net Cashflow',
    'Cumulative Net CF'
]

for r in R:
    ncf_pd_dflist.append(pd.DataFrame(np.transpose(ncf_arr_packed[r])))
ncf_pd = pd.concat(ncf_pd_dflist)
ncf_pd.columns = columns

# Add a date column and reorder
columns.insert(2, 'Date')
ncf_pd['Date'] = pd.Timestamp(BaseDate) + ncf_pd['Month'].apply(lambda m: pd.DateOffset(months=m))
ncf_pd = ncf_pd[columns]

In [10]:
# Truncate pandas dataframe
monthly_out = 60 # months of cash flow output to export
ncf_pd_trunc = ncf_pd.query(f'Month < {monthly_out}')
ncf_pd_trunc.to_csv('monthly_ncf.csv')

In [15]:
# loop though wells in array and create oneline output of economic metrics
# type: ignore
oneline_cat = prop_list.iloc[:, :9]
disc_rate = [0.05, 0.08, 0.09, 0.1, 0.12, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
propnum = []
ror = []
life = []
payout = []
roi = []
droi = []
result_nparr = np.empty([6, 0])
npv_list = []

for r in R:
    propID = prop_list.loc[r, 'PROPNUM']
    wi = prop_list.loc[r, 'WI']
    capex = prop_list.loc[r, 'CAPITAL']
    ncf_r = np.extract(ncf_arr_packed[r][0] == propID, ncf_arr_packed[r][15])
    month_r = np.extract(ncf_arr_packed[r][0] == propID, ncf_arr_packed[r][1])
    cum_ocf_r = np.extract(ncf_arr_packed[r][0] == propID, ncf_arr_packed[r][14])
    cum_ncf_r = np.extract(ncf_arr_packed[r][0] == propID, ncf_arr_packed[r][16])
    
    # calculate npv at all discount rates
    pv_calc = lambda i: npv(i, ncf_r, month_r.astype(int))
    pv = list(map(pv_calc, disc_rate))
    npv_list.append(pv)
    
    # calculate irr
    if wi == 0 or capex == 0:
        irr = max(max(disc_rate), 1)
    else:
        f = lambda x: npv(x, ncf_r, month_r)
        r = optimize.root(f, [0])
        irr = np.clip(r.x[0], 0, max(max(disc_rate), 1))
    ror.append(irr)
    
    # calculate life
    try:
        life_calc = np.clip(np.where(cum_ocf_r == np.max(cum_ocf_r))[0][0] + eloss, 0, duration)
    except:
        life_calc = 1 + eloss
    life.append(np.round(life_calc / 12, 2))
        
    # calculate payout
    ncf_cum = cum_ncf_r[:life_calc]
    month_arr = month_r[:life_calc]
    try:
        payout_interp = np.interp(0, ncf_cum, month_arr)
        payout_calc = payout_interp / 12
    except:
        payout_calc = life_calc / 12
    payout.append(np.round(payout_calc, 2))
        
    # calculate ROI and DROI
    net_capex = capex * weight * inv_wt * wi
    if net_capex == 0:
        roi_calc = 0
        droi_calc = 0
    else:
        roi_calc = np.round(((net_capex + np.sum(ncf_r)) / net_capex), 2)
        droi_calc = np.round(((net_capex + pv[3]) / net_capex), 2)
    roi.append(roi_calc)
    droi.append(droi_calc)
    
    propnum.append(propID)
    oneline_nparr = np.array((propnum, ror, life, payout, roi, droi))

result_nparr = np.column_stack((result_nparr, oneline_nparr))
npv_pd = pd.DataFrame(npv_list, columns = disc_rate)

In [16]:
# Create oneline output export to csv
result_pd = pd.DataFrame(np.transpose(result_nparr), columns = ['UID', 'IRR', 'Life', 'Payout', 'ROI', 'DROI'])
result_pd = pd.merge(result_pd, npv_pd, how = 'inner', left_index = True, right_index = True)
result_pd.to_csv('multiwell_dcf_oneline.csv')

In [17]:
# Calculate breakevens and add to pandas output
# type: ignore
equiv_ratio = 20
disc1 = 0.1
disc2 = 0.2
pri = 50 # breakeven price guess
pajgas = -0.25 # gas differential
run_breakevens = 0

if run_breakevens == 0:
    pass

else:
    # Create function for slicing the volume array and calculating the monthly cash flow
    def econ_be(pri, paj_gas, disc, equiv_ratio, r):    
        econ_be = econ_cf(
            index = r,
            uid = prop_list.loc[r, 'PROPNUM'],
            wi = 1, 
            nri = 0.875,
            roy = prop_list.loc[r, 'Royalty'],
            eloss = eloss, 
            weight = weight,
            prod_wt = prod_wt,
            inv_wt = inv_wt, 
            shrink = np.round(prop_list.loc[r, 'SHRINK'] / 100, 6), 
            btu = np.round(prop_list.loc[r, 'BTU'] / 1000, 6), 
            ngl_yield = np.round(prop_list.loc[r, 'NGL/GAS'], 6), 
            pri_oil = pri, 
            pri_gas = pri / equiv_ratio, 
            paj_oil = prop_list.loc[r, 'PAJ_OIL'], 
            paj_gas = pajgas, 
            paj_ngl = prop_list.loc[r, 'PAJ_NGL'],
            stx_oil = stx_oil,
            stx_gas = stx_gas,
            stx_ngl = stx_ngl,
            adval = adval,
            opc_fix = np.round(prop_list.loc[r, 'OPC/T'], 2), 
            opc_oil = np.round(prop_list.loc[r, 'OIL_OPEX'], 2), 
            opc_gas = np.round(prop_list.loc[r, 'GAS_OPEX'], 2), 
            capex = np.round(prop_list.loc[r, 'CAPITAL'] * 1000, 2),
            aban = aban
        )
        pv_beo = npv(disc, econ_be[15], econ_be[1])
        return pv_beo

    # calculate breakeven prices
    def econ_be_iter1(r):
        pv_beo1 = lambda p: econ_be(p, pajgas, disc1, equiv_ratio, r)
        r_beo1 = optimize.root(pv_beo1, pri)
        return round(r_beo1.x[0], 2)

    vpv_beo1 = np.vectorize(econ_be_iter1)
    beo1 = vpv_beo1(R)

    def econ_be_iter2(r):
        pv_beo2 = lambda p: econ_be(p, pajgas, disc2, equiv_ratio, r)
        r_beo2 = optimize.root(pv_beo2, pri)
        return round(r_beo2.x[0], 2)

    vpv_beo2 = np.vectorize(econ_be_iter2)
    beo2 = vpv_beo2(R)

    # Add results to oneline output
    result_pd['Oil_BE1'] = beo1
    result_pd['Oil_BE2'] = beo2
    result_pd.to_csv('multiwell_dcf_oneline.csv')